In [1]:
""" import """
import os
from dotenv import load_dotenv

import openai

from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import TransformChain, SimpleSequentialChain

In [2]:
""" set OpenAI API key """
load_dotenv(dotenv_path="../../../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(openai_api_key=openai.api_key, temperature=0.9)
chat = ChatOpenAI(openai_api_key=openai.api_key, temperature=0.9)

In [3]:
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()

In [4]:
"""
Generic Transformation Chain

takes in a super long text, filters the text to only the first 3 paragraphs, and then passes that into an LLMChain to summarize those.
"""
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [6]:
template = """Summarize this text:

{output_text}

Summary:"""

prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=llm, prompt=prompt)
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [7]:
sequential_chain.run(state_of_the_union)

' This text is an address to the American people, noting the challenges faced last year due to the COVID-19 pandemic and the importance of coming together as Americans, regardless of political affiliation.'